### Deep Learning Classifier (Melinda Xiao-Devins)

Implement deep learning modle


In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [12]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
 warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
 print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")

#print(dataframe.loc[:,:])
num_rows = df.shape[0]
num_col = df.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))

dataset = df.values
# split into input (X) and output (Y) variables
X = dataset[:,1:num_col]
Y = dataset[:,0]

Total number of records: 376516
Toatl numver of features: 38


In [4]:
#Split into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
#Create model

#create a sequencial model and add layers one at a time
model = Sequential()

# Create a fully connected network with 3 layers
#input layer, it has 12 neurons, it must have right number of inputs, which is the number of features
model.add(Dense(12, input_dim = num_col-1, activation = 'relu'))

# hideen layer has 8 neurons
model.add(Dense(8, activation = 'relu'))

# output layer has 1 neuron to predict
# Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
model.add(Dense(1, activation='sigmoid')) 

In [6]:
#Compile model

# loss function: logarithmic loss, which is binary_crossentropy for binary classification
# use 'adam' optimizer for gradient descent algorithm 
# collect accuracy during training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# Train the model
# epochs: a fixed number of iterations through the dataset
# batch size: the number of instances that are evaluated before a weight update in the network is performed 
model.fit(X_train, Y_train, epochs = 10, batch_size=100)

Epoch 1/10
301212/301212 [==============================] - 12s 39us/step - loss: 0.3676 - acc: 0.8069
Epoch 2/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3426 - acc: 0.8215
Epoch 3/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3404 - acc: 0.8231
Epoch 4/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3394 - acc: 0.8236
Epoch 5/10
301212/301212 [==============================] - 10s 34us/step - loss: 0.3385 - acc: 0.8245
Epoch 6/10
301212/301212 [==============================] - 10s 34us/step - loss: 0.3380 - acc: 0.8243
Epoch 7/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3377 - acc: 0.8249
Epoch 8/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3368 - acc: 0.8255
Epoch 9/10
301212/301212 [==============================] - 10s 33us/step - loss: 0.3357 - acc: 0.8266
Epoch 10/10
301212/301212 [==============================] - 10s 33us/ste

In [8]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

75304/75304 [==============================] - 3s 40us/step

acc: 82.90%


In [9]:
# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded)


the prediction bw. 0 and 1
[[ 0.49364993]
 [ 1.        ]
 [ 0.56724077]
 ..., 
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
the rounded prediction
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

We are going to use scikit-learn to evaluate the model using stratified k-fold cross validation. This is a resampling technique that will provide an estimate of the performance of the model. It does this by splitting the data into k-parts, training the model on all parts except one which is held out as a test set to evaluate the performance of the model. This process is repeated k-times and the average score across all constructed models is used as a robust estimate of performance. It is stratified, meaning that it will look at the output values and attempt to balance the number of instances that belong to each class in the k-splits of the data.

To use Keras models with scikit-learn, we must use the KerasClassifier wrapper. This class takes a function that creates and returns our neural network model. It also takes arguments that it will pass along to the call to fit() such as the number of epochs and the batch size.

In [10]:
# larger model
def create_model():
    # Create a fully connected network with 5 layers
    model = Sequential()
    
    #input layer, it has 64 neurons, it must have right number of inputs, which is the number of features
    model.add(Dense(128, input_dim=num_col-1, kernel_initializer='normal', activation='relu'))
    
    # hideen layer has 64 neurons
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
        
    # hideen layer has 32 neurons
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
        
     # hideen layer has 32 neurons
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
  
    # output layer has 1 neuron to predict
    # Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3393 - acc: 0.8296
Epoch 2/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3222 - acc: 0.8390
Epoch 3/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3192 - acc: 0.8407
Epoch 4/10
338864/338864 [==============================] - 14s 40us/step - loss: 0.3180 - acc: 0.8418
Epoch 5/10
338864/338864 [==============================] - 14s 42us/step - loss: 0.3166 - acc: 0.8430
Epoch 6/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3157 - acc: 0.8436
Epoch 7/10
338864/338864 [==============================] - 14s 42us/step - loss: 0.3150 - acc: 0.8442
Epoch 8/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3140 - acc: 0.8447
Epoch 9/10
338864/338864 [==============================] - 14s 41us/step - loss: 0.3134 - acc: 0.8452
Epoch 10/10
37652/37652 [==============================] - 1s 16us/step
E

338865/338865 [==============================] - 14s 42us/step - loss: 0.3158 - acc: 0.8430
Epoch 7/10
338865/338865 [==============================] - 14s 42us/step - loss: 0.3150 - acc: 0.8440
Epoch 8/10
338865/338865 [==============================] - 14s 42us/step - loss: 0.3144 - acc: 0.8444
Epoch 9/10
338865/338865 [==============================] - 15s 43us/step - loss: 0.3135 - acc: 0.8450
Epoch 10/10
37651/37651 [==============================] - 1s 20us/step
Epoch 1/10
338865/338865 [==============================] - 15s 45us/step - loss: 0.3391 - acc: 0.8286
Epoch 2/10
338865/338865 [==============================] - 15s 43us/step - loss: 0.3232 - acc: 0.8383
Epoch 3/10
338865/338865 [==============================] - 14s 42us/step - loss: 0.3196 - acc: 0.8405
Epoch 4/10
338865/338865 [==============================] - 14s 43us/step - loss: 0.3176 - acc: 0.8421
Epoch 5/10
338865/338865 [==============================] - 15s 43us/step - loss: 0.3166 - acc: 0.8433
Epoch 6/10
3